Importing necesary libraries

In [36]:
print(np.__version__)

2.0.1


In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#%matplotlib inline

In [11]:
import warnings
warnings.filterwarnings('ignore')
#Modellong
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBRegressor
#from catboost import CatBoostRegressor


In [12]:
data=pd.read_csv('data\stud.csv')

In [13]:
data.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [14]:
X=data.drop(columns=['math_score'],axis=1)

In [15]:
y=data['math_score']

In [16]:
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features),
    ]
)

In [17]:
X=preprocessor.fit_transform(X)

In [19]:
#SEPARATE DATA INTO TRAIN AND TEST
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [22]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2_square=r2_score(true,predicted)
    return mae,mse,rmse,r2_square

In [25]:
models={
    "Linear Regression":LinearRegression(),
    "Lasso Regression": Lasso(),
    "Ridge Regression": Ridge(),
    "K-Neighbours Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #make predictions
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #Evaluate train and test dataset
    model_train_mae,model_train_mse,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for training set')
    print(" Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print(" Mean Absolute Error: {:.4f}".format(model_train_mae))
    print(" R2 SCORE: {:.4f}".format(model_train_r2))


    print('---------------------------------------------------------------')

    print('Model Performance for testset')
    print(" Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print(" Mean Absolute Error: {:.4f}".format(model_test_mae))
    print(" R2 SCORE: {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)
    print('=================================================================')
    print('\n')

Linear Regression
Model Performance for training set
 Root Mean Squared Error: 5.2674
 Mean Absolute Error: 4.2075
 R2 SCORE: 0.8742
---------------------------------------------------------------
Model Performance for testset
 Root Mean Squared Error: 5.6330
 Mean Absolute Error: 4.4779
 R2 SCORE: 0.8725


Lasso Regression
Model Performance for training set
 Root Mean Squared Error: 6.5106
 Mean Absolute Error: 5.1582
 R2 SCORE: 0.8078
---------------------------------------------------------------
Model Performance for testset
 Root Mean Squared Error: 6.8705
 Mean Absolute Error: 5.3929
 R2 SCORE: 0.8103


Ridge Regression
Model Performance for training set
 Root Mean Squared Error: 5.2487
 Mean Absolute Error: 4.1977
 R2 SCORE: 0.8751
---------------------------------------------------------------
Model Performance for testset
 Root Mean Squared Error: 5.5566
 Mean Absolute Error: 4.4155
 R2 SCORE: 0.8759


K-Neighbours Regressor
Model Performance for training set
 Root Mean Square

In [27]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','R2 Score']).sort_values(by=["R2 Score"],ascending=False)

,Model Name,R2 Score
2,Ridge Regression,0.875907
0,Linear Regression,0.872469
5,Random Forest Regressor,0.849873
7,AdaBoost Regressor,0.839447
6,XGBRegressor,0.811816
1,Lasso Regression,0.810281
3,K-Neighbours Regressor,0.772528
4,Decision Tree,0.723779


LINEAR REGRESSION

In [28]:
lin_model=LinearRegression(fit_intercept=True)
lin_model=lin_model.fit(X_train,y_train)
y_pred=lin_model.predict(X_test)
score=r2_score(y_test,y_pred)*100
print("Accuracy of the model is %.2f" %score)

Accuracy of the model is 87.25


Plot Y PRED AND y TEST

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')

Difference between actual and predicted values

In [40]:
pred_df=pd.DataFrame({'Actual Value': y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df

,Actual Value,Predicted Value,Difference
521,91,76.125,14.875
737,53,59.500,-6.500
740,80,77.125,2.875
660,74,76.750,-2.750
411,84,88.500,-4.500
...,...,...,...
468,77,73.250,3.750
935,70,62.750,7.250
428,65,58.375,6.625
7,40,44.500,-4.500
